In [39]:
import pandas as pd
import glob

path = "LoLesports_data"

all_files = glob.glob(path + "/*.csv")

df_list = []

for file in all_files[-3:]:
    temp_df = pd.read_csv(file, low_memory=False)
    df_list.append(temp_df)

df = pd.concat(df_list, ignore_index=True)
df = df[df["datacompleteness"] == "complete"]
df.drop(columns=["datacompleteness", "url", "year"], inplace=True)

train = df[(df["patch"] >= 12.01) & (df["patch"] <= 14.05)]
test = df[df["patch"] >= 14.06]

train.shape, test.shape

((266040, 158), (67380, 158))

In [40]:
players = train[train["position"] != "team"].reset_index(drop=True)
players.shape

(221700, 158)

In [42]:
# 선수별 챔피언 통계 계산
player_champ_stats = (
    players.groupby(["playername", "champion"])
    .agg(
        {
            "result": ["count", "mean"],
            "kills": "mean",
            "deaths": "mean",
            "assists": "mean",
            "dpm": "mean",
            "damageshare": "mean",
            "vspm": "mean",
            "earnedgoldshare": "mean",
        }
    )
    .reset_index()
)

# 컬럼명 정리
player_champ_stats.columns = [
    "player",
    "champion",
    "games_played",
    "winrate",
    "avg_kills",
    "avg_deaths",
    "avg_assists",
    "avg_dpm",
    "avg_dmg_share",
    "avg_vspm",
    "avg_gold_share",
]

# 픽률 계산
total_games_by_player = players.groupby("playername")["result"].count().reset_index()
total_games_by_player.columns = ["player", "total_games"]

player_champ_stats = player_champ_stats.merge(total_games_by_player, on="player")
player_champ_stats["pickrate"] = (
    player_champ_stats["games_played"] / player_champ_stats["total_games"]
)

# 최소 5게임 이상 플레이한 데이터만 필터링
player_champ_stats = player_champ_stats[player_champ_stats["games_played"] >= 5]

# 승률과 픽률 기준으로 정렬
player_champ_stats.sort_values(["winrate", "pickrate"], ascending=False, inplace=True)

In [44]:
player_champ_stats["avg_kda"] = (
    player_champ_stats["avg_kills"] + player_champ_stats["avg_assists"]
) / player_champ_stats["avg_deaths"]

player_champ_stats.sort_values("games_played", ascending=False, inplace=True)
player_champ_stats

,player,champion,games_played,winrate,avg_kills,avg_deaths,avg_assists,avg_dpm,avg_dmg_share,avg_vspm,avg_gold_share,total_games,pickrate,avg_kda
56561,unknown player,Ornn,92,0.369565,1.543478,3.228261,6.119565,416.935920,0.197723,0.731866,0.186134,2858,0.032190,2.373737
56554,unknown player,Nautilus,90,0.422222,0.677778,4.400000,8.033333,158.480983,0.073887,2.335367,0.091847,2858,0.031491,1.979798
56574,unknown player,Sejuani,83,0.457831,1.939759,3.277108,7.698795,385.766553,0.169780,1.013502,0.178267,2858,0.029041,2.941176
56606,unknown player,Viego,69,0.434783,3.840580,3.333333,6.000000,355.884622,0.164376,1.189032,0.199493,2858,0.024143,2.952174
56526,unknown player,Kai'Sa,68,0.367647,4.750000,3.308824,5.294118,528.043472,0.238938,0.941722,0.265809,2858,0.023793,3.035556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12972,Emets,Kai'Sa,5,0.600000,7.600000,1.800000,4.800000,557.414960,0.289176,1.362540,0.301412,74,0.067568,6.888889
27868,Lyng,Renata Glasc,5,0.600000,0.400000,2.200000,7.800000,150.961200,0.072164,2.427300,0.086198,74,0.067568,3.727273
52056,Yuta,Nocturne,5,0.600000,2.600000,3.600000,9.600000,408.297720,0.159864,1.558560,0.188674,74,0.067568,3.388889
759,Acerola,Sejuani,5,0.600000,1.600000,2.200000,8.000000,272.707200,0.146175,1.474160,0.169724,75,0.066667,4.363636


In [46]:
player_champ_stats = player_champ_stats[
    player_champ_stats["player"] != "unknown player"
]
player_champ_stats

,player,champion,games_played,winrate,avg_kills,avg_deaths,avg_assists,avg_dpm,avg_dmg_share,avg_vspm,avg_gold_share,total_games,pickrate,avg_kda
34948,Peyz,Aphelios,62,0.677419,4.693548,1.983871,4.516129,621.644540,0.298079,1.338677,0.275794,246,0.252033,4.642276
10391,Delight,Rakan,59,0.644068,0.728814,1.881356,9.288136,129.493553,0.066892,3.566834,0.097626,261,0.226054,5.324324
14312,Faker,Azir,57,0.649123,2.684211,2.368421,4.842105,654.819895,0.293048,1.234049,0.233713,318,0.179245,3.177778
33461,Oner,Lee Sin,55,0.763636,3.927273,2.581818,7.236364,343.745176,0.151613,1.462816,0.190135,334,0.164671,4.323944
52451,Zeka,Azir,53,0.566038,3.226415,2.000000,4.094340,624.685438,0.297099,1.354540,0.259980,286,0.185315,3.660377
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8485,Cozy,Fiora,5,0.600000,3.200000,1.800000,3.200000,599.545340,0.297888,1.237800,0.248121,74,0.067568,3.555556
12972,Emets,Kai'Sa,5,0.600000,7.600000,1.800000,4.800000,557.414960,0.289176,1.362540,0.301412,74,0.067568,6.888889
27868,Lyng,Renata Glasc,5,0.600000,0.400000,2.200000,7.800000,150.961200,0.072164,2.427300,0.086198,74,0.067568,3.727273
52056,Yuta,Nocturne,5,0.600000,2.600000,3.600000,9.600000,408.297720,0.159864,1.558560,0.188674,74,0.067568,3.388889
